In [1]:
import torch , numpy as np, torch_geometric, random, pickle
from deephyper.problem import HpProblem
import ray
from tqdm import tqdm

2024-01-15 18:05:03.162280: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 18:05:03.200168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 18:05:03.200192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 18:05:03.201097: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 18:05:03.207084: I tensorflow/core/platform/cpu_feature_guar

In [2]:
torch.cuda._device_count_nvml()

2

In [2]:
class AMDGCNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = torch_geometric.nn.GATConv(16, 32, edge_dim=2)
        self.gcn2 = torch_geometric.nn.GATConv(32, 32, edge_dim=2)
        self.gcn3 = torch_geometric.nn.GATConv(32, 32, edge_dim=2)
        self.gcn4 = torch_geometric.nn.GATConv(32, 1, edge_dim=2)
        self.global_pool = torch_geometric.nn.aggr.SortAggregation(k=30)
        self.conv1 = torch.nn.Conv1d(1, 16, 97, 97)
        self.conv2 = torch.nn.Conv1d(16, 32, 5, 1)
        self.maxpool = torch.nn.MaxPool1d(2, 2)
        self.linear1 = torch.nn.Linear(352, 128)
        self.linear2 = torch.nn.Linear(128, 1)
        self.dropout = torch.nn.Dropout(0.5)
    def forward(self, mat, edge_index):
        h1 = self.gcn1(mat, edge_index).tanh()
        h2 = self.gcn2(h1, edge_index).tanh()
        h3 = self.gcn3(h2, edge_index).tanh()
        h4 = self.gcn4(h3, edge_index).tanh()
        h = torch.cat([h1, h2, h3, h4], dim=-1)
        h = self.global_pool(h)
        h = h.view(h.size(0), 1, h.size(-1))
        h = self.conv1(h).relu()
        h = self.maxpool(h)
        h = self.conv2(h).relu()
        h = h.view(h.size(0), -1)
        h = self.linear1(h).relu()
        h = self.dropout(h)
        h = self.linear2(h).sigmoid()
        return h

In [3]:
def train_one_epoch(k, model, optimizer, criterion, device):
    running_loss = 0.
    last_loss = 0.
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    pos_batch_dat = np.load(f'pos_batch_{k}.npz', allow_pickle=True)
    neg_batch_dat = np.load(f'neg_batch_{k}.npz', allow_pickle=True)
    pos_edge_dat = np.load(f'pos_edge_attr_{k}.npz', allow_pickle=True)
    neg_edge_dat = np.load(f'neg_edge_attr_{k}.npz', allow_pickle=True)
    pos_table = pos_batch_dat.f.arr_0
    neg_table = neg_batch_dat.f.arr_0
    pos_edges = pos_edge_dat.f.arr_0
    neg_edges = neg_edge_dat.f.arr_0
    lab_t = [j for j in range(len(pos_table)+len(neg_table)-1)]
    random.shuffle(lab_t)
    print(f"Batch {k} in progress")
    ind = 0
    for i in lab_t:
        label = None
        g = None
        x = None
        e = None
        if i < len(pos_table):
            label = torch.tensor([1.], dtype=torch.float32).to(device)
            g = pos_table[i][2]
            x = pos_table[i][3]
            e = pos_edges[i]
        else:
            label = torch.tensor([0.], dtype=torch.float32).to(device)
            g = neg_table[i - len(pos_table)][2]
            x = neg_table[i - len(pos_table)][3]
            e = neg_edges[i - len(pos_table)]
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        inp = torch.sparse_coo_tensor(torch.tensor(g, dtype=torch.float32).to_sparse_coo().indices(), torch.tensor(e, dtype = torch.float32)).coalesce().to(device)
        outputs = model( torch.tensor(x).to(torch.float32).to(device), inp)
        # Compute the loss and its gradients
        loss = criterion(outputs.view(1), label)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        ind +=1
        running_loss += loss.item()
        if ind == 197:
            last_loss = running_loss # loss per batch
            print(f'Output: {outputs}, label: {label}')
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.
    return last_loss

In [5]:
def get_acc(model, device):
    #device = torch.device("cuda")
    torch.no_grad()
    model.train(False)
    actuals = []
    expecteds = []
    for k in range(31,40):
        pos_batch_dat = np.load(f'pos_batch_{k}.npz', allow_pickle=True)
        neg_batch_dat = np.load(f'neg_batch_{k}.npz', allow_pickle=True)
        pos_edge_dat = np.load(f'pos_edge_attr_{k}.npz', allow_pickle=True)
        neg_edge_dat = np.load(f'neg_edge_attr_{k}.npz', allow_pickle=True)
        pos_table = pos_batch_dat.f.arr_0
        neg_table = neg_batch_dat.f.arr_0
        pos_edges = pos_edge_dat.f.arr_0
        neg_edges = neg_edge_dat.f.arr_0
        lab_t = [j for j in range(len(pos_table)+len(neg_table)-1)]
        random.shuffle(lab_t)
        print(f"Batch {k} in progress")
        ind = 0
        outs = []
        temp_l = []
        for i in lab_t:
            label = None
            g = None
            x = None
            e = None
            if i < len(pos_table):
                temp_l.append(1)
                g = pos_table[i][2]
                x = pos_table[i][3]
                e = pos_edges[i]
            else:
                temp_l.append(0)
                g = neg_table[i - len(pos_table)][2]
                x = neg_table[i - len(pos_table)][3]
                e = neg_edges[i - len(pos_table)]

            # Make predictions for this batch
            inp = torch.sparse_coo_tensor(torch.tensor(g, dtype=torch.float32).to_sparse_coo().indices(), torch.tensor(e, dtype = torch.float32)).coalesce().to(device)
            outputs = model( torch.tensor(x).to(torch.float32).to(device), inp)
            o = 0
            if outputs.view(1)[0] > 0.5:
                o = 1
            outs.append(o)
            torch.cuda.empty_cache()
        expecteds.extend(temp_l)
        actuals.extend(outs)
        torch.cuda.empty_cache()
    sum = 0.
    for l in range(len(expecteds)):
        if expecteds[l] == actuals[l]:
            sum += 1
    return sum/len(actuals)



In [6]:
def run(job):
    device = torch.device("cuda")
    model = AMDGCNN()
    model.train(True)
    model.to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr = job["learning_rate"])
    criterion = torch.nn.BCELoss()
    for epoch in range(job["num_epochs"]):
        for i in range(1, 31):
            train_one_epoch(i, model, optimizer, criterion, device)
    return get_acc(model, device)


In [7]:
problem = HpProblem()

# Discrete hyperparameter (sampled with uniform prior)
problem.add_hyperparameter((5, 20), "num_epochs", default_value=10)

# Discrete and Real hyperparameters (sampled with log-uniform)
problem.add_hyperparameter((0.0001, 0.1, "log-uniform"), "learning_rate", default_value=0.0005)

learning_rate, Type: UniformFloat, Range: [0.0001, 0.1], Default: 0.0005, on log-scale

In [8]:
# We launch the Ray run-time and execute the `run` function
# with the default configuration
n_gpus = torch.cuda.device_count()
is_gpu_available = torch.cuda.is_available()
if is_gpu_available:
    if not(ray.is_initialized()):
        ray.init(num_cpus=n_gpus, num_gpus=n_gpus, log_to_driver=False)
else:
    if not(ray.is_initialized()):
        ray.init(num_cpus=1, log_to_driver=False)

2024-01-05 10:44:27,710	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [9]:
from deephyper.evaluator import Evaluator
from deephyper.evaluator.callback import TqdmCallback

def get_evaluator(run_function):
    # Default arguments for Ray: 1 worker and 1 worker per evaluation
    method_kwargs = {
        "num_cpus": 1,
        "num_cpus_per_task": 1,
        "callbacks": [TqdmCallback()]
    }

    # If GPU devices are detected then it will create 'n_gpus' workers
    # and use 1 worker for each evaluation
    if torch.cuda.is_available():
        method_kwargs["num_cpus"] = n_gpus
        method_kwargs["num_gpus"] = n_gpus
        method_kwargs["num_cpus_per_task"] = 1
        method_kwargs["num_gpus_per_task"] = 1

    evaluator = Evaluator.create(
        run_function,
        method="ray",
        method_kwargs=method_kwargs
    )
    print(f"Created new evaluator with {evaluator.num_workers} worker{'s' if evaluator.num_workers > 1 else ''} and config: {method_kwargs}", )

    return evaluator

evaluator_1 = get_evaluator(run)

Created new evaluator with 2 workers and config: {'num_cpus': 2, 'num_cpus_per_task': 1, 'callbacks': [<deephyper.evaluator.callback.TqdmCallback object at 0x7f851c55d550>], 'num_gpus': 2, 'num_gpus_per_task': 1}


/home/dp1200@unt.ad.unt.edu/.venv/lib64/python3.9/site-packages/deephyper/evaluator/_evaluator.py:127: UserWarning: Applying nest-asyncio patch for IPython Shell!
  warnings.warn(


In [10]:
from deephyper.search.hps import CBO
# Uncomment the following line to show the arguments of CBO.
# CBO?
# Instanciate the search with the problem and a specific evaluator
search = CBO(problem, evaluator_1)

In [11]:
results = search.search(max_evals=30)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
pickle.dump(results, 'results.pickle')

NameError: name 'results' is not defined

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('results.csv')
data

,p:learning_rate,p:num_epochs,objective,job_id,m:timestamp_submit,m:timestamp_gather
0,0.000562,8,0.979131,0,1.465135,6849.214205
1,0.004340,15,0.497462,1,1.466548,12405.218339
2,0.075996,14,0.502538,2,6849.245531,18422.994636
3,0.001667,14,0.502538,3,12405.243189,24205.495469
4,0.005337,9,0.502538,4,18423.019018,26079.387467
5,0.002732,6,0.502538,6,26079.413719,31268.952611
6,0.000497,6,0.521715,7,31268.976658,36478.823181
7,0.000229,17,0.561760,5,24205.525767,38487.672269
8,0.000569,12,0.959955,8,36478.849388,46586.471162
